In [56]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>

<article class="day-desc read-aloud"><h2>--- Day 9: Smoke Basin ---</h2><p>These caves seem to be <a href="https://en.wikipedia.org/wiki/Lava_tube" target="_blank">lava tubes</a>. Parts are even still volcanically active; small hydrothermal vents release smoke into the caves that slowly <span title="This was originally going to be a puzzle about watersheds, but we're already under water.">settles like rain</span>.</p>
<p>If you can model how the smoke flows through the caves, you might be able to avoid it and be that much safer. The submarine generates a heightmap of the floor of the nearby caves for you (your puzzle input).</p>
<p>Smoke flows to the lowest point of the area it's in. For example, consider the following heightmap:</p>
<pre><code>2<em>1</em>9994321<em>0</em>
3987894921
98<em>5</em>6789892
8767896789
989996<em>5</em>678
</code></pre>
<p>Each number corresponds to the height of a particular location, where <code>9</code> is the highest and <code>0</code> is the lowest a location can be.</p>
<p>Your first goal is to find the <em>low points</em> - the locations that are lower than any of its adjacent locations. Most locations have four adjacent locations (up, down, left, and right); locations on the edge or corner of the map have three or two adjacent locations, respectively. (Diagonal locations do not count as adjacent.)</p>
<p>In the above example, there are <em>four</em> low points, all highlighted: two are in the first row (a <code>1</code> and a <code>0</code>), one is in the third row (a <code>5</code>), and one is in the bottom row (also a <code>5</code>). All other locations on the heightmap have some lower adjacent location, and so are not low points.</p>
<p>The <em>risk level</em> of a low point is <em>1 plus its height</em>. In the above example, the risk levels of the low points are <code>2</code>, <code>1</code>, <code>6</code>, and <code>6</code>. The sum of the risk levels of all low points in the heightmap is therefore <code><em>15</em></code>.</p>
<p>Find all of the low points on your heightmap. <em>What is the sum of the risk levels of all low points on your heightmap?</em></p>
</article>


In [57]:
from collections import deque
from math import prod
from collections.abc import Generator, Iterator


example = """
2199943210
3987894921
9856789892
8767896789
9899965678
"""


class HeightMap:
    ADJACENT = (-1, 0), (0, 1), (1, 0), (0, -1)

    def __init__(self, s: str) -> None:
        map = [[int(i) for i in l] for l in s.strip().splitlines()]
        self.map = (
            [[10] * (len(map[0]) + 2)]
            + [[10] + l + [10] for l in map]
            + [[10] * (len(map[0]) + 2)]
        )
        self.rows, self.cols = len(map), len(map[0])

    def lowpoints(self) -> Iterator[tuple[int, int]]:
        yield from (
            (r, c)
            for r, c in product(range(1, self.rows + 1), range(1, self.cols + 1))
            if all(self[r, c] < self[r + dr, c + dc] for dr, dc in self.ADJACENT)
        )

    def adjacent(self, r, c) -> Iterator[tuple[int, int]]:
        yield from (
            (r + dr, c + dc)
            for dr, dc in self.ADJACENT
            if 0 <= r + dr < len(self.map) and 0 <= c + dc < len(self.map[0])
        )

    def basins(self) -> list[list[tuple[int, int]]]:
        basins = [[lp] for lp in self.lowpoints()]

        for basin in basins:
            queue = deque([basin.pop()])
            seen = set()

            while queue:
                r, c = queue.popleft()

                if self[r, c] > 8 or (r, c) in seen:
                    continue

                seen.add((r, c))
                basin.append((r, c))
                queue.extend(self.adjacent(r, c))

        return basins

    def three_largest_basins_and_multiply_sizes(self) -> int:  # -> Any:
        return prod(sorted(len(b) for b in self.basins())[-3:])

    def __getitem__(self, key: tuple[int, int]) -> int:
        r, c = key
        return self.map[r][c]

    def __repr__(self) -> str:
        return "\n".join("".join(f"{i:2}" for i in l) for l in self.map[1:-1])


def lowpoints(s: str) -> int:
    heightmap = HeightMap(s)
    return sum(heightmap[r, c] + 1 for r, c in heightmap.lowpoints())


assert lowpoints((example)) == 15

In [58]:
with open("../input/day9.txt") as f:
    puzzle = f.read()

print(f"Part I: {lowpoints(puzzle)}")

Part I: 436


<link href="style.css" rel="stylesheet"></link>

<main>

<p>Your puzzle answer was <code>436</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>Next, you need to find the largest basins so you know what areas are most important to avoid.</p>
<p>A <em>basin</em> is all locations that eventually flow downward to a single low point. Therefore, every low point has a basin, although some basins are very small. Locations of height <code>9</code> do not count as being in any basin, and all other locations will always be part of exactly one basin.</p>
<p>The <em>size</em> of a basin is the number of locations within the basin, including the low point. The example above has four basins.</p>
<p>The top-left basin, size <code>3</code>:</p>
<pre><code><em>21</em>99943210
<em>3</em>987894921
9856789892
8767896789
9899965678
</code></pre>
<p>The top-right basin, size <code>9</code>:</p>
<pre><code>21999<em>43210</em>
398789<em>4</em>9<em>21</em>
985678989<em>2</em>
8767896789
9899965678
</code></pre>
<p>The middle basin, size <code>14</code>:</p>
<pre><code>2199943210
39<em>878</em>94921
9<em>85678</em>9892
<em>87678</em>96789
9<em>8</em>99965678
</code></pre>
<p>The bottom-right basin, size <code>9</code>:</p>
<pre><code>2199943210
3987894921
9856789<em>8</em>92
876789<em>678</em>9
98999<em>65678</em>
</code></pre>
<p>Find the three largest basins and multiply their sizes together. In the above example, this is <code>9 * 14 * 9 = <em>1134</em></code>.</p>
<p><em>What do you get if you multiply together the sizes of the three largest basins?</em></p>
</article>

</main>


In [59]:
assert HeightMap(example).three_largest_basins_and_multiply_sizes() == 1134

In [60]:
print(f"Part II: { HeightMap(puzzle).three_largest_basins_and_multiply_sizes()}")

Part II: 1317792


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>1317792</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
